# Analyze
Where you can analyze an audio file.

In [53]:
import math

import torch
from pydub import AudioSegment
import pandas as pd
from IPython.display import display

from cnn.models import Model
from data.import_data import load_dataset, get_dataloader, load_file
from eval import eval_model

In [49]:
# utility functions

def play_second(audio, second):
    '''
    Plays the audio for a given second of the file
    '''

    return audio[second * 1000: (second + 1) * 1000]

def calculate_deepfake_value(preds):
    '''
    Calculates the deepfake value from the model predictions
    '''

    # square the predictions
    preds_squared = [pred ** 2 for pred in preds]

    # calculate the average of the squared predictions
    avg = sum(preds_squared) / len(preds_squared)

    # return the square root of the average
    return math.sqrt(avg)

In [50]:
# get the evaluation data, sorted to show the longest clips

eval_data = pd.read_csv('data/meta/eval.csv')
eval_data.sort_values('length (sec)', ascending=False).head(20)

,file,speaker,label,numeric_label,length (sec)
2441,21311.wav,Alec Guinness,spoof,1,24.727000
1659,14627.wav,Alec Guinness,spoof,1,24.645063
2714,23557.wav,Alec Guinness,bona-fide,0,23.437062
712,6404.wav,Alec Guinness,spoof,1,22.530062
1208,10661.wav,Alec Guinness,spoof,1,22.126000
1702,15026.wav,Alec Guinness,spoof,1,21.874000
1909,16669.wav,Alec Guinness,spoof,1,20.028000
2346,20505.wav,Alec Guinness,spoof,1,18.667000
3365,29289.wav,Alec Guinness,spoof,1,18.566063
2272,19868.wav,Alec Guinness,spoof,1,18.541000


In [71]:
# set the paths to the file you want to analyze, and the model
file_path = 'data_files/14627.wav'
model_path = 'models/1.pth'

# load the model
model = Model(num_channels=1)
model.load_state_dict(torch.load(model_path))

# load the file
file_tensors = load_file(file_path)

# run the clip through the model
model.eval()

with torch.no_grad():
    model_preds = model(torch.stack(file_tensors))[:, 0]

# sort the seconds by their deepfake probability
probs_sorted = sorted([(i, prob.item()) for i, prob in enumerate(model_preds)], key=lambda x: x[1], reverse=True)

# calculate the deepfake value
## (avg of the squared predictions, square rooted, to give more weight to higher probabilities)
deepfake_prob = calculate_deepfake_value(model_preds)

In [73]:
for i, prob in probs_sorted:
    print(f'Second {i}, Probability: {prob}')

Second 20, Probability: 0.5592638254165649
Second 10, Probability: 0.5390245914459229
Second 15, Probability: 0.5354292392730713
Second 11, Probability: 0.5231657028198242
Second 5, Probability: 0.5224992036819458
Second 19, Probability: 0.5206447839736938
Second 24, Probability: 0.5196725726127625
Second 23, Probability: 0.5105296969413757
Second 9, Probability: 0.500051736831665
Second 13, Probability: 0.4973704218864441
Second 12, Probability: 0.4893817603588104
Second 16, Probability: 0.4785928726196289
Second 4, Probability: 0.4756999611854553
Second 3, Probability: 0.46586474776268005
Second 17, Probability: 0.4545885920524597
Second 22, Probability: 0.4390139877796173
Second 21, Probability: 0.438944011926651
Second 6, Probability: 0.43776777386665344
Second 8, Probability: 0.4297375977039337
Second 18, Probability: 0.42109110951423645
Second 0, Probability: 0.41909655928611755
Second 2, Probability: 0.41577181220054626
Second 14, Probability: 0.4099285304546356
Second 1, Probab

## Display Results

In [74]:
print(f'This clip has a deepfake probability of {round(deepfake_prob * 100, 2)}%\n')
print('The most likely deepfake seconds are:')
for i, prob in probs_sorted[:5]:
    print(f'    Second {i}: {prob:.2f}')
print('')


# load the audio
audio = AudioSegment.from_file(file_path)

# display the most likely deepfake seconds
for i, prob in probs_sorted[:5]:
    print(f'Second {i}:')
    display(play_second(audio, i))

This clip has a deepfake probability of 47.13%

The most likely deepfake seconds are:
    Second 20: 0.56
    Second 10: 0.54
    Second 15: 0.54
    Second 11: 0.52
    Second 5: 0.52

Second 20:


Second 10:


Second 15:


Second 11:


Second 5:
